In [1]:
cd ../src/

/Users/brent/Archiver/Workspace/qcqp/src


In [7]:
import collections


import numpy as np
import pandas as pd

###########
# display options
###########


pd.set_option("display.max_columns", None)
np.set_printoptions(
  linewidth=200,
  precision=4
)

from pyqp import bg_grb, bg_msk, bg_msk_msc, bg_msk_chordal, bg_msk_homo
from pyqp import bb_msc, bb, bb_diag, bb_socp
from pyqp.classes import QP, QPI, Bounds, BCParams
import argparse
import json

METHODS = collections.OrderedDict([
  ("grb", bg_grb.qp_gurobi),
  ("shor", bg_msk.shor),
  ("dshor", bg_msk.dshor),
  ("msc", bg_msk_msc.msc),
  ("emsc", bg_msk_msc.msc_diag),
  ("ssdp", bg_msk_chordal.ssdp),
  ("bb", bb.bb_box),
  ("bb_msc", bb_diag.bb_box),
  ("shor_homo", bg_msk_homo.shor)
])


METHOD_CODES = {
  idx + 1: m
  for idx, m in enumerate(METHODS)
}

METHOD_HELP_MSGS = {
  k: bg_msk.dshor.__doc__
  for k, v in METHODS.items()
}

parser = argparse.ArgumentParser("QCQP runner")
parser.add_argument("--dump_instance", type=int, help="if save instance", default=1)
parser.add_argument("--r", type=str, help=METHOD_CODES.__str__(), default="1,2,7")
parser.add_argument("--fpath", type=str, help="path of the instance")
parser.add_argument("--n", type=int, help="dim of x", default=5)
parser.add_argument("--m", type=int, help="if randomly generated num of constraints", default=5)
parser.add_argument("--pc", type=str, help="if randomly generated problem type", default=5)
parser.add_argument("--time_limit", default=60, type=int, help="time limit of running.")
parser.add_argument("--verbose", default=0, type=int, help="if verbose")
parser.add_argument("--bg", default='msk', type=str, help="backend used")
parser.add_argument("--bg_rd", default=0, type=int, help="backend used, rank reduction")
parser.add_argument("-f", default=0, type=str, help="backend used, rank reduction")


_StoreAction(option_strings=['-f'], dest='f', nargs=None, const=None, default=0, type=<class 'str'>, choices=None, help='backend used, rank reduction', metavar=None)

In [3]:
parser.print_usage()
args = parser.parse_args()
params = BCParams()
kwargs, r_methods = params.produce_args(parser, METHOD_CODES)

usage: QCQP runner [-h] [--dump_instance DUMP_INSTANCE] [--r R]
                   [--fpath FPATH] [--n N] [--m M] [--pc PC]
                   [--time_limit TIME_LIMIT] [--verbose VERBOSE] [--bg BG]
                   [--bg_rd BG_RD] [-f F]
usage: QCQP runner [-h] [--dump_instance DUMP_INSTANCE] [--r R]
                   [--fpath FPATH] [--n N] [--m M] [--pc PC]
                   [--time_limit TIME_LIMIT] [--verbose VERBOSE] [--bg BG]
                   [--bg_rd BG_RD] [-f F]


In [8]:
r_methods = {'bb', 'grb', 'bb_msc'}

In [5]:
n, m = 10, 2
# problem
problem_id = f"{n}:{m}:{0}"
# start
# qp = QPI.block(n, m, r=2, eps=0.5)
qp = QPI.normal(int(n), int(m), rho=0.2)
bd = Bounds(xlb=np.zeros(shape=(n, 1)), xub=np.ones(shape=(n, 1)))

# benchmarking

In [9]:
evals = []
results = {}
# run methods
for k in r_methods:
  func = METHODS[k]
  qp1 = bb_msc.QP(*qp.unpack())
  qp1.decompose()
  r = func(qp1, bd, params=params)
  reval = r.eval(problem_id)
  evals.append({**reval.__dict__, "method": k})
  results[k] = r

Using license file /Users/brent/licenses/gurobi.lic
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 1 threads
Optimize a model with 0 rows, 10 columns and 0 nonzeros
Model fingerprint: 0x468b86cb
Model has 26 quadratic objective terms
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 3e+01]
  QLMatrix range   [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [3e+01, 3e+01]

Continuous model is non-convex 

In [11]:
results['grb'].xval, results['bb_msc'].xval.round(2)

(array([[0.   ],
        [0.124],
        [1.   ],
        [1.   ],
        [0.   ],
        [0.   ],
        [1.   ],
        [1.   ],
        [0.   ],
        [1.   ]]),
 array([[-0.  ],
        [ 0.27],
        [ 1.  ],
        [ 1.  ],
        [-0.  ],
        [-0.  ],
        [ 1.  ],
        [ 1.  ],
        [-0.  ],
        [ 1.  ]]))

In [12]:
for k, r in results.items():
  print(f"{k} benchmark @{r.relax_obj}")
  r.check(qp)

grb benchmark @31.062496185302734
bb benchmark @31.062499931943485
bb_msc benchmark @31.119824257337324


In [13]:
xval = results['grb'].xval
xval

array([[0.   ],
       [0.124],
       [1.   ],
       [1.   ],
       [0.   ],
       [0.   ],
       [1.   ],
       [1.   ],
       [0.   ],
       [1.   ]])

In [17]:
from pyqp.bg_msk_msc import msc_diag, msc
from pyqp.bg_msk_msc import dom, expr, mf

In [49]:
qp1 = bb_msc.QP(*qp.unpack())
qp1.decompose(decompose_method='eig-type2')

In [50]:
r = msc_diag(qp1, solve=False)

In [51]:
# model 
model = r.problem
x = r.xvar  
y = r.yvar 
z = r.zvar

In [52]:
for yy in r.Yvar:
  sumy = expr.sum(yy)
  model.constraint(expr.sub(expr.dot(x, results['grb'].xval), sumy), dom.greaterThan(0))
model.constraint(expr.sub(expr.dot(x, results['grb'].xval), (results['grb'].xval**2).sum()), dom.greaterThan(0))

mosek.fusion.LinearConstraint

In [53]:
r.solve()

Problem
  Name                   : many_small_cone_msk
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 69              
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 30              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   : many_small_cone_msk
  Objective sense        : max             
  Type                   : CONIC (conic opt

In [54]:
r.xval, xval

(array([[-0.   ],
        [ 0.124],
        [ 1.   ],
        [ 1.   ],
        [-0.   ],
        [-0.   ],
        [ 1.   ],
        [ 1.   ],
        [ 0.   ],
        [ 1.   ]]),
 array([[0.   ],
        [0.124],
        [1.   ],
        [1.   ],
        [0.   ],
        [0.   ],
        [1.   ],
        [1.   ],
        [0.   ],
        [1.   ]]))

In [48]:
r.xval.T @ xval - xval.T@xval

array([[3.9281e-05]])

In [44]:
(r.Zval ** 2).sum(0) - r.Yval.sum(0)

array([-0.385 , -0.3258, -0.3252])

In [43]:
(r.Zval ** 2).sum(0) - (r.xval ** 2).sum(0)

array([-1.8430e-05, -9.2740e-05, -1.5192e-04])